In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))

In [24]:
import pickle
from sklearn.cluster import KMeans
from scripts.data_visualiser import *
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.feature_extraction import *
from model_scripts.get_statistics import *
from Pipeline.pre_processing_pipeline import *
import numpy as np
import preprocessing_config as config

In [14]:
temporal_images = load_field_images_temporal(config.base_directory_temporal)

In [15]:
len(temporal_images)

48

In [ ]:
np.unique(temporal_images[0][0][:,:,-1])

array([       0., 20190616.])

### Clustering Trial - NDVI DIff

In [ ]:
ndvi_images = temporal_differences_with_time(temporal_images,'ndvi')
len(ndvi_images)

9

In [ ]:
ndvi_images[0][0].shape

(64, 64)

In [ ]:
ndvi_tensors = []

for stack in ndvi_images:
    ndvi_tensor = np.stack(stack, axis=-1)
    ndvi_tensors.append(ndvi_tensor)

print("NDVI Tensor Shape:", ndvi_tensors[0].shape)

NDVI Tensor Shape: (64, 64, 6)


In [ ]:
# # Reshape to N x T
# h, w, t = ndvi_tensor.shape
# flattened_tensor = ndvi_tensor.reshape(-1, t)
# print("Flattened Tensor Shape:", flattened_tensor.shape)  # Should be (4096, 6)

In [ ]:
# features = []
# for img in ndvi_tensors:
#     mean_features = np.mean(img, axis=(0, 1))  # Mean across spatial dimensions
#     std_features = np.std(img, axis=(0, 1))    # Std across spatial dimensions
#     min_features = np.min(img, axis=(0, 1))    # Min values across spatial dimensions
#     max_features = np.max(img, axis=(0, 1))    # Max values across spatial dimensions
    
#     # Concatenate features (mean, std, min, max for each temporal slice)
#     img_features = np.concatenate([mean_features, std_features, min_features, max_features])
#     features.append(img_features)

# features = np.array(features)
# print("Feature Matrix Shape:", features.shape)  # Should be (#images, 24) if T=6

In [ ]:
features = np.array([img.flatten() for img in ndvi_tensors])
print("Feature Matrix Shape:", features.shape)       # (#images, 64*64*6)

Feature Matrix Shape: (9, 24576)


In [ ]:
from sklearn.cluster import KMeans

n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(features)

print("Cluster Labels Shape:", cluster_labels.shape)    # (#images,)

Cluster Labels Shape: (9,)


/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
print(cluster_labels)

[1 0 1 0 1 0 0 0 1]


### Clustering Trial - NDVI and Bands

In [ ]:
len(temporal_images)

9

In [11]:
index_bands = indexbands_temporal_cubes(temporal_images, 'ndvi')

In [12]:
index_bands[0][0].shape

(64, 64, 3)

In [13]:
image_tensors = []

for stack in index_bands:
    image_tensor = np.stack(stack, axis=0)
    image_tensors.append(image_tensor)

print("NDVI Tensor Shape:", image_tensors[0].shape)

NDVI Tensor Shape: (7, 64, 64, 3)


Flattening the 7×64×64×3 stack into a 1D feature vector:

In [ ]:
features = []
for stack in image_tensors:
    feature_vector = stack.flatten()
    features.append(feature_vector)

In [26]:
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(features)

/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [27]:
cluster_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Histogram Features?

In [16]:
features = np.array([compute_histograms(stack) for stack in image_tensors])
print("Feature Matrix Shape:", features.shape)  # Shape: (X, features_per_stack)

Feature Matrix Shape: (9, 30)


In [20]:
features = np.nan_to_num(features, nan=0)

In [21]:
from sklearn.cluster import KMeans

# Apply K-Means
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(features)

print("Cluster Labels Shape:", cluster_labels.shape)  # Shape: (X,)

Cluster Labels Shape: (9,)


/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [22]:
cluster_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Clustering Trial - k-means on single images

In [16]:
simple_images = get_non_temporal_images(temporal_images)

In [17]:
simple_images[0].shape

(64, 64, 13)

In [18]:
refined_images, field_numbers = refine_chanel_non_temporal(simple_images)

In [19]:
refined_images[0].shape

(64, 64, 3)

In [20]:
features = np.array([img.flatten() for img in refined_images])
print("Feature Matrix Shape:", features.shape)    

Feature Matrix Shape: (48, 12288)


In [21]:
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(features)

print("Cluster Labels Shape:", cluster_labels.shape)    # (#images,)

Cluster Labels Shape: (48,)


/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [22]:
cluster_labels

array([1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1], dtype=int32)

In [23]:
field_numbers

['1168039.0',
 '1168039.0_1228889.0',
 '1168663.0_1176271.0',
 '1168692.0_1220431.0',
 '1171179.0_1222688.0',
 '1172001.0',
 '1172001.0_1225692.0',
 '1174805.0',
 '1176318.0',
 '1176365.0',
 '1177274.0',
 '1177490.0',
 '1177490.0_1183928.0',
 '1178038.0',
 '1178198.0_1225298.0',
 '1179855.0_1187336.0',
 '1182776.0',
 '1182822.0_1216964.0',
 '1183171.0',
 '1194806.0_1194808.0',
 '1196327.0',
 '1196347.0',
 '1196489.0_1196747.0',
 '1216230.0_1219006.0',
 '1216239.0',
 '1216250.0',
 '1216801.0_1216804.0',
 '1217032.0',
 '1217498.0',
 '1219929.0_1219932.0',
 '1220322.0',
 '1222641.0',
 '1223341.0',
 '1223970.0',
 '1224018.0',
 '1224356.0',
 '1225081.0',
 '1225397.0',
 '1225547.0',
 '1225592.0',
 '1226125.0',
 '1226225.0',
 '1226904.0_1227074.0',
 '1227071.0',
 '1227321.0_1229711.0',
 '1227495.0',
 '1227663.0',
 '1230818.0']

In [35]:
fn = get_accuracy(field_numbers, cluster_labels, config.labels_path)
fn

0.4262295081967213